# Builder Agent Development

This notebook configures the Builder Agent for generating optimized insurance claims with **structured JSON output**.

**Prerequisites:**
- **MUST run `01_Data_Views_for_Agent_Workflows.ipynb` first**
- Cortex Complete service enabled
- All data foundation tables/views from previous notebook

**What this builds:**
- JSON schema for structured claim output
- Builder Agent configuration with prompts
- Claim generation and optimization logic
- API endpoint definitions
- Data integration for agent queries

**CRITICAL:** Builder Agent enforces structured JSON output - no free text responses allowed.


## Step 1: JSON Schema Definition

Define the structured JSON format that the Builder Agent must output for insurance claims.


In [ ]:
CREATE TABLE IF NOT EXISTS CLAIMS_DEMO.PUBLIC.INSURANCE_CLAIM_SCHEMA (
    SCHEMA_VERSION VARCHAR(10) PRIMARY KEY,
    JSON_SCHEMA_TEXT TEXT,
    SCHEMA_DESCRIPTION TEXT,
    CREATED_DATE TIMESTAMP DEFAULT CURRENT_TIMESTAMP()
);


In [ ]:
snow --config-file ./config.toml sql -q "
INSERT INTO CLAIMS_DEMO.PUBLIC.INSURANCE_CLAIM_SCHEMA (SCHEMA_VERSION, JSON_SCHEMA_TEXT, SCHEMA_DESCRIPTION) VALUES
('v1.0', 
'{\"claim_header\":{\"claim_id\":\"CLM-YYYY-XXXXXX\",\"claim_date\":\"YYYY-MM-DD\",\"patient_id\":\"PAT_XXX\",\"provider_npi\":\"1234567890\",\"insurance_provider\":\"Cigna\",\"policy_number\":\"string\",\"group_number\":\"string\"},\"patient_info\":{\"first_name\":\"string\",\"last_name\":\"string\",\"date_of_birth\":\"YYYY-MM-DD\",\"gender\":\"Male/Female\",\"medical_history\":\"string\",\"allergies\":\"string\",\"current_medications\":\"string\"},\"claim_details\":{\"procedure_codes\":[\"85025\",\"80053\"],\"diagnosis_codes\":[\"I10\",\"E785\"],\"service_date\":\"YYYY-MM-DD\",\"place_of_service\":\"11\",\"clinical_notes\":\"string\",\"medical_necessity_statement\":\"string\"},\"billing_info\":{\"total_charges\":150.00,\"line_items\":[{\"procedure_code\":\"85025\",\"description\":\"Complete Blood Count\",\"quantity\":1,\"unit_charge\":75.00,\"total_charge\":75.00,\"modifiers\":[]}]},\"supporting_documentation\":{\"physician_notes\":\"string\",\"prior_authorization_number\":\"string or null\",\"referral_information\":\"string or null\",\"additional_documentation\":\"string\"}}',
'Standard insurance claim JSON structure for Builder Agent output with all required Cigna fields');
" -c healthcare_claims_demo


## Step 2: Builder Agent Configuration

Configure the Builder Agent with system prompts and **mandatory JSON output** requirements.


In [ ]:
CREATE TABLE IF NOT EXISTS CLAIMS_DEMO.PUBLIC.BUILDER_AGENT_CONFIG (
    CONFIG_ID VARCHAR(20) PRIMARY KEY,
    AGENT_NAME VARCHAR(100),
    SYSTEM_PROMPT TEXT,
    OUTPUT_FORMAT_REQUIREMENT TEXT,
    DATA_SOURCES TEXT,
    CREATED_DATE TIMESTAMP DEFAULT CURRENT_TIMESTAMP()
);


In [ ]:
INSERT INTO CLAIMS_DEMO.PUBLIC.BUILDER_AGENT_CONFIG (CONFIG_ID, AGENT_NAME, SYSTEM_PROMPT, OUTPUT_FORMAT_REQUIREMENT, DATA_SOURCES) VALUES
('BUILDER_V1', 
'Claims Builder Agent',
'You are a Builder Agent specialized in creating optimized insurance claims for Cigna. Your role: 1) Gather patient info, procedure details, clinical notes 2) Generate properly formatted claims optimized for Cigna approval 3) When receiving rebuttal feedback, strengthen claims to address issues. Reference Cigna policies and successful patterns.',
'MANDATORY: Output ONLY valid JSON following the insurance claim schema. No additional text or explanations. JSON must include: claim_header, patient_info, claim_details, billing_info, supporting_documentation.',
'PATIENTS, COMMON_PROCEDURES, CIGNA_POLICY_RULES, AGENT_POLICY_LOOKUP, BUILDER_AGENT_SUCCESS_GUIDE');


## Step 3: Claim Generation Prompts

Create specialized prompts for initial claim generation and iterative optimization with **structured JSON output**.


In [ ]:
CREATE TABLE IF NOT EXISTS CLAIMS_DEMO.PUBLIC.CLAIM_GENERATION_PROMPTS (
    PROMPT_ID VARCHAR(20) PRIMARY KEY,
    PROMPT_TYPE VARCHAR(50),
    PROMPT_TEXT TEXT,
    EXPECTED_OUTPUT_FORMAT TEXT,
    CREATED_DATE TIMESTAMP DEFAULT CURRENT_TIMESTAMP()
);


In [ ]:
INSERT INTO CLAIMS_DEMO.PUBLIC.CLAIM_GENERATION_PROMPTS (PROMPT_ID, PROMPT_TYPE, PROMPT_TEXT, EXPECTED_OUTPUT_FORMAT) VALUES
('INITIAL_CLAIM', 'First Claim Generation',
'Generate an insurance claim for Cigna based on the following provider inputs:

PATIENT: {patient_data}
PROCEDURE: {procedure_data} 
CLINICAL NOTES: {clinical_notes}

REQUIREMENTS:
1. Use patient information to populate demographics and insurance details
2. Reference Cigna policy rules for coverage and requirements
3. Generate appropriate diagnosis codes based on clinical notes and medical history
4. Calculate realistic billing amounts based on procedure cost ranges
5. Include all required documentation per Cigna policy

OUTPUT: Return ONLY a complete JSON object following the insurance claim schema. No additional text.',
'Valid JSON with claim_header, patient_info, claim_details, billing_info, supporting_documentation'),

('REBUTTAL_RESPONSE', 'Claim Optimization After Rebuttal',
'Improve the insurance claim based on Insurance Agent feedback:

ORIGINAL CLAIM: {original_claim_json}
INSURANCE AGENT REBUTTAL: {rebuttal_feedback}
STRENGTH SCORE: {current_strength_score}

OPTIMIZATION REQUIREMENTS:
1. Address each specific issue raised in the rebuttal
2. Strengthen weak areas identified by the Insurance Agent
3. Add missing documentation or authorizations if required
4. Improve medical necessity justification
5. Ensure all Cigna policy requirements are met
6. Maintain claim accuracy while maximizing approval probability

OUTPUT: Return ONLY the improved JSON claim object. No additional text.',
'Enhanced JSON claim addressing all rebuttal feedback with improved strength');


In [ ]:
CREATE TABLE IF NOT EXISTS CLAIMS_DEMO.PUBLIC.AGENT_INTERACTIONS (
    INTERACTION_ID VARCHAR(50) PRIMARY KEY,
    SESSION_ID VARCHAR(50),
    AGENT_TYPE VARCHAR(20), -- 'BUILDER' or 'INSURANCE'
    INPUT_DATA VARIANT,
    OUTPUT_DATA VARIANT,
    INTERACTION_TIMESTAMP TIMESTAMP DEFAULT CURRENT_TIMESTAMP(),
    PROCESSING_STATUS VARCHAR(20), -- 'SUCCESS', 'ERROR', 'PENDING'
    ERROR_MESSAGE TEXT
);


In [ ]:
CREATE OR REPLACE VIEW CLAIMS_DEMO.PUBLIC.BUILDER_AGENT_QUERY_HELPER AS
SELECT 
    'PATIENT_' || p.PATIENT_ID as QUERY_KEY,
    OBJECT_CONSTRUCT(
        'patient_id', p.PATIENT_ID,
        'name', p.FIRST_NAME || ' ' || p.LAST_NAME,
        'insurance', p.INSURANCE_PROVIDER,
        'policy', p.POLICY_NUMBER,
        'medical_history', p.MEDICAL_HISTORY_SUMMARY,
        'allergies', p.ALLERGIES,
        'medications', p.CURRENT_MEDICATIONS
    ) as PATIENT_DATA,
    NULL as PROCEDURE_DATA,
    NULL as POLICY_DATA
FROM CLAIMS_DEMO.PUBLIC.PATIENTS p

UNION ALL

SELECT 
    'PROCEDURE_' || cp.PROCEDURE_CODE as QUERY_KEY,
    NULL as PATIENT_DATA,
    OBJECT_CONSTRUCT(
        'code', cp.PROCEDURE_CODE,
        'name', cp.PROCEDURE_NAME,
        'specialty', cp.MEDICAL_SPECIALTY,
        'cost_range', cp.TYPICAL_COST_RANGE,
        'coverage_notes', cp.CIGNA_COVERAGE_NOTES
    ) as PROCEDURE_DATA,
    NULL as POLICY_DATA
FROM CLAIMS_DEMO.PUBLIC.COMMON_PROCEDURES cp

UNION ALL

SELECT 
    'POLICY_' || cpr.POLICY_RULE_ID as QUERY_KEY,
    NULL as PATIENT_DATA,
    NULL as PROCEDURE_DATA,
    OBJECT_CONSTRUCT(
        'rule_id', cpr.POLICY_RULE_ID,
        'category', cpr.RULE_CATEGORY,
        'procedures', cpr.PROCEDURE_CODES,
        'coverage', cpr.COVERAGE_CRITERIA,
        'prior_auth', cpr.PRIOR_AUTH_REQUIRED,
        'requirements', cpr.MEDICAL_NECESSITY_REQUIREMENTS
    ) as POLICY_DATA
FROM CLAIMS_DEMO.PUBLIC.CIGNA_POLICY_RULES cpr;


In [ ]:
CREATE OR REPLACE VIEW CLAIMS_DEMO.PUBLIC.BUILDER_AGENT_DATA_ACCESS AS
SELECT 
    'PATIENT_LOOKUP' as DATA_TYPE,
    p.PATIENT_ID,
    p.FIRST_NAME,
    p.LAST_NAME,
    p.INSURANCE_PROVIDER,
    p.POLICY_NUMBER,
    p.MEDICAL_HISTORY_SUMMARY,
    p.ALLERGIES,
    p.CURRENT_MEDICATIONS,
    NULL as PROCEDURE_CODE,
    NULL as PROCEDURE_NAME,
    NULL as COVERAGE_NOTES
FROM CLAIMS_DEMO.PUBLIC.PATIENTS p

UNION ALL

SELECT 
    'PROCEDURE_REFERENCE' as DATA_TYPE,
    NULL as PATIENT_ID,
    NULL as FIRST_NAME,
    NULL as LAST_NAME,
    NULL as INSURANCE_PROVIDER,
    NULL as POLICY_NUMBER,
    NULL as MEDICAL_HISTORY_SUMMARY,
    NULL as ALLERGIES,
    NULL as CURRENT_MEDICATIONS,
    cp.PROCEDURE_CODE,
    cp.PROCEDURE_NAME,
    cp.CIGNA_COVERAGE_NOTES
FROM CLAIMS_DEMO.PUBLIC.COMMON_PROCEDURES cp;


## Step 3: API Endpoint Definitions

Define REST API endpoints for frontend integration with **structured JSON input/output**.


In [ ]:
CREATE TABLE IF NOT EXISTS CLAIMS_DEMO.PUBLIC.BUILDER_AGENT_API (
    ENDPOINT_ID VARCHAR(20) PRIMARY KEY,
    ENDPOINT_NAME VARCHAR(100),
    HTTP_METHOD VARCHAR(10),
    ENDPOINT_URL VARCHAR(200),
    REQUEST_FORMAT TEXT,
    RESPONSE_FORMAT TEXT,
    CORTEX_INTEGRATION TEXT,
    CREATED_DATE TIMESTAMP DEFAULT CURRENT_TIMESTAMP()
);


In [ ]:
snow --config-file ./config.toml sql -q "
INSERT INTO CLAIMS_DEMO.PUBLIC.BUILDER_AGENT_API (ENDPOINT_ID, ENDPOINT_NAME, HTTP_METHOD, ENDPOINT_URL, REQUEST_FORMAT, RESPONSE_FORMAT, CORTEX_INTEGRATION) VALUES
('BUILD_INITIAL', 'Generate Initial Claim', 'POST', '/api/v1/builder/generate-claim',
'Input JSON: {\"patient_id\": \"PAT_XXX\", \"procedure_code\": \"XXXXX\", \"clinical_notes\": \"string\"}',
'Output JSON: Complete insurance claim following schema with claim_header, patient_info, claim_details, billing_info, supporting_documentation',
'Uses Cortex Complete with BUILDER_V1 system prompt + patient/procedure/policy data context'),

('BUILD_OPTIMIZE', 'Optimize Claim After Rebuttal', 'POST', '/api/v1/builder/optimize-claim',
'Input JSON: {\"original_claim\": {claim_json}, \"rebuttal_feedback\": \"string\", \"strength_score\": number}',
'Output JSON: Enhanced insurance claim with improvements addressing rebuttal feedback',
'Uses Cortex Complete with REBUTTAL_RESPONSE prompt + original claim + feedback context');
" -c healthcare_claims_demo


## Step 4: Sample JSON Output

Create a sample JSON claim to demonstrate the **exact structured format** the Builder Agent must produce.


In [ ]:
snow --config-file ./config.toml sql -q "
CREATE OR REPLACE TABLE CLAIMS_DEMO.PUBLIC.SAMPLE_CLAIM_OUTPUT AS
SELECT 
    'SAMPLE_001' as SAMPLE_ID,
    'PAT_001' as PATIENT_ID,
    '85025' as PROCEDURE_CODE,
    PARSE_JSON('{
        \"claim_header\": {
            \"claim_id\": \"CLM-2024-001234\",
            \"claim_date\": \"2024-09-17\",
            \"patient_id\": \"PAT_001\",
            \"provider_npi\": \"1234567890\",
            \"insurance_provider\": \"Cigna\",
            \"policy_number\": \"CGN123456789\",
            \"group_number\": \"CIGNA_GRP001\"
        },
        \"patient_info\": {
            \"first_name\": \"John\",
            \"last_name\": \"Smith\",
            \"date_of_birth\": \"1975-03-15\",
            \"gender\": \"Male\",
            \"medical_history\": \"Hypertension, Type 2 Diabetes\",
            \"allergies\": \"Penicillin\",
            \"current_medications\": \"Metformin 500mg, Lisinopril 10mg\"
        },
        \"claim_details\": {
            \"procedure_codes\": [\"85025\"],
            \"diagnosis_codes\": [\"I10\", \"E11\"],
            \"service_date\": \"2024-09-17\",
            \"place_of_service\": \"11\",
            \"clinical_notes\": \"Annual wellness screening for diabetic patient\",
            \"medical_necessity_statement\": \"CBC required for monitoring diabetes management and hypertension treatment effectiveness\"
        },
        \"billing_info\": {
            \"total_charges\": 75.00,
            \"line_items\": [{
                \"procedure_code\": \"85025\",
                \"description\": \"Complete Blood Count (CBC)\",
                \"quantity\": 1,
                \"unit_charge\": 75.00,
                \"total_charge\": 75.00,
                \"modifiers\": []
            }]
        },
        \"supporting_documentation\": {
            \"physician_notes\": \"Patient presents for annual wellness exam. Diabetes and hypertension well controlled.\",
            \"prior_authorization_number\": null,
            \"referral_information\": null,
            \"additional_documentation\": \"Previous lab results from 2023 for comparison\"
        }
    }') as CLAIM_JSON;
" -c healthcare_claims_demo


## Step 5: Verification

Verify the Builder Agent is properly configured with **structured JSON output**.


In [ ]:
-- Verify sample claim JSON structure is accessible
SELECT 
    SAMPLE_ID,
    CLAIM_JSON:claim_header:claim_id as CLAIM_ID,
    CLAIM_JSON:patient_info:first_name as PATIENT_NAME,
    CLAIM_JSON:claim_details:procedure_codes as PROCEDURES,
    CLAIM_JSON:billing_info:total_charges as TOTAL_CHARGES
FROM CLAIMS_DEMO.PUBLIC.SAMPLE_CLAIM_OUTPUT;


## Step 7: Final Setup - Test Input and Verification

Create test input scenarios and verify the complete Builder Agent setup.


In [ ]:
CREATE OR REPLACE TABLE CLAIMS_DEMO.PUBLIC.BUILDER_AGENT_TEST_INPUT AS
SELECT 
    'TEST_SESSION_001' as SESSION_ID,
    'PAT_001' as PATIENT_ID,
    '85025' as PROCEDURE_REQUESTED,
    'Patient John Smith presents with fatigue and requests routine blood work. No acute symptoms. Annual wellness screening.' as CLINICAL_NOTES,
    CURRENT_TIMESTAMP() as REQUEST_TIMESTAMP;


In [ ]:
-- FINAL VERIFICATION: Builder Agent complete setup validation  
-- All counts should be 1 for successful setup
SELECT 
    COUNT(CASE WHEN table_name = 'INSURANCE_CLAIM_SCHEMA' THEN 1 END) as HAS_SCHEMA,
    COUNT(CASE WHEN table_name = 'BUILDER_AGENT_CONFIG' THEN 1 END) as HAS_CONFIG,
    COUNT(CASE WHEN table_name = 'CLAIM_GENERATION_PROMPTS' THEN 1 END) as HAS_PROMPTS,
    COUNT(CASE WHEN table_name = 'BUILDER_AGENT_API' THEN 1 END) as HAS_API,
    COUNT(CASE WHEN table_name = 'SAMPLE_CLAIM_OUTPUT' THEN 1 END) as HAS_SAMPLE,
    COUNT(CASE WHEN table_name = 'AGENT_INTERACTIONS' THEN 1 END) as HAS_LOGGING
FROM CLAIMS_DEMO.INFORMATION_SCHEMA.TABLES 
WHERE table_schema = 'PUBLIC';

-- Verify Builder Agent views exist
SELECT table_name as VIEW_NAME
FROM CLAIMS_DEMO.INFORMATION_SCHEMA.VIEWS 
WHERE table_schema = 'PUBLIC' 
    AND table_name IN ('BUILDER_AGENT_QUERY_HELPER', 'BUILDER_AGENT_DATA_ACCESS')
ORDER BY table_name;


In [ ]:
-- SUCCESS CONFIRMATION: Builder Agent ready for Cortex integration
SELECT 
    'BUILDER_AGENT_SETUP_COMPLETE' as STATUS,
    'Ready for Insurance Agent Development (Phase 4)' as NEXT_PHASE,
    'Structured JSON Output Enforced' as JSON_STATUS,
    'All API endpoints defined' as API_STATUS;
